## Домашнее задание

### Урок 1. Инсталлы. Стартовая воронка

#### Выполнил: Илья Корнев

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

table_schema = 'data_viz_1068'
table_name = 'project_dataset'

## Задачи

+ Проанализируйте динамику количества ежедневных инсталлов в игре для органики и
неорганики. Какие особенности изменений этих показателей вы видите? Сгенерируйте
гипотезы, почему могут наблюдаться такие изменения (которые затем в ходе выполнения
дальнейших заданий вы сможете проверить).
+ Есть ли отличия этого показателя для разных регионов (СНГ, Европа, Азия, Северная Америка и Канада, Южная Америка)? Сделайте оценку в каких регионах более качественное
удержание на начальном этапе (можно оценить, например, по второй сессии и проценту
игроков, запустивших игру повторно).
+ Проанализируйте поведение игроков в первые 24 часа после первого запуска игры.
Какие события совершают пользователи чаще всего? Есть ли паттерны поведения
игроков разных когорт (по типу пользователей, регионам и т.д.)?

## Предобработка данных

In [ ]:
project_dataset = pd.read_sql(f'''
    SELECT *
    FROM {table_schema}.project_dataset
''', con=engine)
project_dataset

,event_time,event_date,event_name,revenue_usd,region,country,device_type,platform,cohort_date,user_id,user_type,content_id,currency,revenue
0,2020-11-29 20:09:47,2020-11-29,LaunchApp,0.0,EU,RU,iPad 6,ios,2020-10-28,176269,non_organic,None,None,0.0
1,2020-11-29 20:09:49,2020-11-29,LaunchApp,0.0,EU,RU,OnePlus-ONEPLUS A6010,android,2020-11-15,187611,organic,None,None,0.0
2,2020-11-29 20:09:51,2020-11-29,LaunchApp,0.0,EU,RU,xiaomi-Redmi Note 7,android,2020-10-25,169714,non_organic,None,None,0.0
3,2020-11-29 20:09:53,2020-11-29,LaunchApp,0.0,EU,RU,Xiaomi-Redmi 8A,android,2020-11-11,199466,organic,None,None,0.0
4,2020-11-29 20:09:56,2020-11-29,LaunchApp,0.0,AS,KZ,HUAWEI-PAR-LX1,android,2020-11-29,207932,organic,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176993,2020-11-29 20:09:28,2020-11-29,achieve_level_5,0.0,EU,RU,HONOR-KSA-LX9,android,2020-11-27,207554,organic,None,None,0.0
1176994,2020-11-29 20:09:28,2020-11-29,LaunchApp,0.0,EU,UA,Xiaomi-MI MAX 2,android,2020-11-01,184562,organic,None,None,0.0
1176995,2020-11-29 20:09:30,2020-11-29,LaunchApp,0.0,EU,RU,asus-ASUS_X00TD,android,2020-11-01,184331,organic,None,None,0.0
1176996,2020-11-29 20:09:35,2020-11-29,achieve_level_4,0.0,EU,UA,motorola-moto e6s,android,2020-11-26,207209,organic,None,None,0.0


In [ ]:
# Удаление дубликатов:
project_dataset = project_dataset.drop_duplicates().reset_index(drop=True)

In [ ]:
# Обработка даты:
project_dataset['event_time'] = pd.to_datetime(project_dataset['event_time'], format='%Y-%m-%d %H:%M:%S')
project_dataset['event_date'] = pd.to_datetime(project_dataset['event_date'], format='%Y-%m-%d')
project_dataset['cohort_date'] = pd.to_datetime(project_dataset['cohort_date'], format='%Y-%m-%d')

### Подзадача

Узнать, существуют ли в базе данных события запуска приложения, записанные более ранним событием, чем "FirstLaunchApp".

#### Расчёты

In [ ]:
# Собираю все первые запуски в одну таблицу:
installs = project_dataset[project_dataset['event_name'] == 'FirstLaunchApp'].reset_index(drop=True)

In [ ]:
# Проверяю дубликаты по первому запуску игры:
installs['user_id'].duplicated().sum()

12460

В данных 12 460 записей с событием "первого запуска приложения" для одних и тех же пользователей.

Поскольку **первое** событие должно быть только одно для каждого пользователя, решаю убрать дубликаты группировкой событий с использованием минимальных значений event_time.

In [ ]:
installs_compare = installs.groupby(['event_name', 'user_id']).agg({'event_time': 'min'}).reset_index().sort_values('event_time').reset_index(drop=True)
installs_compare.head(2)

,event_name,user_id,event_time
0,FirstLaunchApp,100024,2020-10-22 10:12:31
1,FirstLaunchApp,100025,2020-10-22 10:26:37


In [ ]:
# Проверка дубликатов после обработки данных:
installs_compare['user_id'].duplicated().sum()

0

In [ ]:
# Собираю все запуски приложения во вторую таблицу:
launches = project_dataset[project_dataset['event_name'] == 'LaunchApp'].loc[:,
    ('event_time', 'event_name', 'user_id')].reset_index(drop=True).sort_values(['event_time']).reset_index(drop=True)
launches.head(2)

,event_time,event_name,user_id
0,2020-10-22 10:12:31,LaunchApp,100024
1,2020-10-22 10:26:37,LaunchApp,100025


In [ ]:
# Сопоставляю события первого запуска и всех запусков приложения:
launches_compare = pd.merge(installs_compare, launches, how='left',on=['user_id'])
launches_compare

,event_name_x,user_id,event_time_x,event_time_y,event_name_y
0,FirstLaunchApp,100024,2020-10-22 10:12:31,2020-10-22 10:12:31,LaunchApp
1,FirstLaunchApp,100025,2020-10-22 10:26:37,2020-10-22 10:26:37,LaunchApp
2,FirstLaunchApp,100025,2020-10-22 10:26:37,2020-10-22 11:34:01,LaunchApp
3,FirstLaunchApp,100025,2020-10-22 10:26:37,2020-10-22 11:35:37,LaunchApp
4,FirstLaunchApp,100025,2020-10-22 10:26:37,2020-10-22 11:36:36,LaunchApp
...,...,...,...,...,...
683714,FirstLaunchApp,232720,2021-05-31 12:33:20,NaT,NaN
683715,FirstLaunchApp,190441,2021-05-31 13:20:42,NaT,NaN
683716,FirstLaunchApp,232722,2021-05-31 14:15:21,NaT,NaN
683717,FirstLaunchApp,232723,2021-05-31 14:20:52,NaT,NaN


In [ ]:
# Проверяю, существуют ли события запуска приложения с более ранним временем, чем первый запуск:
(launches_compare['event_time_y'] < launches_compare['event_time_x']).sum()

73757

Таким образом, после удаления повторяющихся событий FirstLaunchApp, время запуска приложения (LaunchApp) записывалось более ранними событиями, чем FirstLaunchApp, 73 757 раз в случаях, когда FirstLaunchApp для пользователя записан.

При этом существуют пользователи, для которых не существует события FirstLaunchApp. Это может быть связано с рядом причин:
+ Первый запуск приложения пользователем состоялся раньше, чем имеющиеся у нас данные;
+ Данные являются неполными по другим причинам;
+ В базе данных нет отдельной категории тестеров, однако данные по ним, как правило, существенно отличаются от средних.

In [ ]:
# Если сделать merge по LaunchApp, можно увидеть игроков с событиями запуска приложения, но без события первого запуска:
no_first_launch = pd.merge(installs, launches, how='right',on=['user_id'])
no_first_launch.tail(5)

,event_time_x,event_date,event_name_x,revenue_usd,region,country,device_type,platform,cohort_date,user_id,user_type,content_id,currency,revenue,event_time_y,event_name_y
5123085,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,120902,NaN,NaN,NaN,NaN,2021-05-31 15:49:17,LaunchApp
5123086,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,136613,NaN,NaN,NaN,NaN,2021-05-31 16:01:10,LaunchApp
5123087,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,137667,NaN,NaN,NaN,NaN,2021-05-31 16:25:01,LaunchApp
5123088,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,137667,NaN,NaN,NaN,NaN,2021-05-31 16:31:20,LaunchApp
5123089,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,168718,NaN,NaN,NaN,NaN,2021-05-31 17:49:37,LaunchApp


In [ ]:
# Пользователь 120902 относится к когорте 2020-09-29, а данные в нашем датасете начинаются с 2020-10-22:
project_dataset[(project_dataset['user_id'] == 120902)].sort_values('event_time').reset_index(drop=True).head(1)

,event_time,event_date,event_name,revenue_usd,region,country,device_type,platform,cohort_date,user_id,user_type,content_id,currency,revenue
0,2020-11-11 12:27:54,2020-11-11,LaunchApp,0.0,EU,RU,Xiaomi-Mi 9 Lite,android,2020-09-29,120902,non_organic,None,None,0.0


In [ ]:
# Пользователь 168718 относится к когорте 2020-10-25, то есть должен входить в наши данные, но при этом не имеет события FirstLauchApp, а первый же взятый им уровень - 9-й:
project_dataset[(project_dataset['user_id'] == 168718)].sort_values('event_time').reset_index(drop=True).head(6)

,event_time,event_date,event_name,revenue_usd,region,country,device_type,platform,cohort_date,user_id,user_type,content_id,currency,revenue
0,2020-11-20 17:53:46,2020-11-20,LaunchApp,0.0,EU,RU,samsung-SM-J400F,android,2020-10-25,168718,non_organic,None,None,0.0
1,2020-11-21 09:29:50,2020-11-21,LaunchApp,0.0,EU,RU,samsung-SM-J400F,android,2020-10-25,168718,non_organic,None,None,0.0
2,2020-11-22 03:52:42,2020-11-22,LaunchApp,0.0,EU,RU,samsung-SM-J400F,android,2020-10-25,168718,non_organic,None,None,0.0
3,2020-11-22 10:18:20,2020-11-22,LaunchApp,0.0,EU,RU,samsung-SM-J400F,android,2020-10-25,168718,non_organic,None,None,0.0
4,2020-12-04 14:58:46,2020-12-04,LaunchApp,0.0,EU,RU,samsung-SM-J400F,android,2020-10-25,168718,non_organic,None,None,0.0
5,2020-12-04 15:28:43,2020-12-04,achieve_level_9,0.0,EU,RU,samsung-SM-J400F,android,2020-10-25,168718,non_organic,None,None,0.0


### Выводы

+ В БД большое количество некорректных данных, событие инсталла не является первым для большого числа пользователей, 73,7 тысяч запусков приложения произошло **до** события, считающимся первым запуском в наших данных. В этом случае у меня есть три варианта будущей работы с данными:
 - Выбрать ID пользователей, для которых у нас есть FirstLaunchApp, отказаться от уже имеющихся данных по FirstLaunchApp и назначить события самого раннего LaunchApp в качестве FirstLaunchApp. Мне кажется, это интересное решение, но оно может оказаться некорректным, внести ещё больше хаоса в данные и сделать их ещё менее пригодными для анализа.
 - Отсечь всех пользователей, у которых обычные запуски приложения случились до первого, поскольку эти данные некорректны и могут исказить анализ данных.
 - Не делать ничего. Сначала этот вариант кажется неприличным, однако, в процессе работы при таком решении часть некорректных данных всё равно будет отсеиваться, например, при подсчёте удержания все запуски, случившиеся до первого, будут выдавать отрицательный лайфтайм и убираться.
+ Действие, которое стоило бы сделать независимо от решения по тому, как работать с данными - написание баг-репорта для дата инженеров об имеющихся проблемах данных.
+ Если для пользователя отсутствует FirstLaunchApp, его данные являются неполными на основании нашей базы данных. У таких пользователей в первых же имеющихся событиях может произойти взятие уровня выше 4-го, то есть это либо тестеры, которые могут выдать себе нужный уровень, либо игроки, игравшие раньше, но не имеющие записи о первом запуске приложения в нашей базе данных.

## Расчёт метрик для анализа

### Задача 1. Динамика количества ежедневных инсталлов для органики и неорганики

#### Расчёты

In [ ]:
# Убираю дубликаты по пользователям, сохраняя тип пользователей:
installs_usertype = installs.drop_duplicates(['user_id']).reset_index(drop=True).loc[:, ('event_date', 'user_id', 'user_type')].reset_index(drop=True)

In [ ]:
# Группирую данные инсталлов по типу трафика и дате:
installs_usertype = installs_usertype.groupby(['user_type', 'event_date'])['user_id'].nunique().reset_index().sort_values('event_date').reset_index(drop=True)

In [ ]:
# Переименовываю user_id в user_count:
installs_usertype.rename(columns={'user_id': 'user_count'}, inplace=True)

#### График

In [ ]:
# Получаю столбчатый график с визуализацией количества инсталлов пользователями из разных типов трафика по датам:
fig = px.bar(
    installs_usertype,
    x = 'event_date',
    y = 'user_count',
    color = 'user_type',
    title = 'Распределение инсталлов по типам трафика',
)
fig.show()

#### Гипотезы
+ В 20-х числах октября 2020 года в игре что-то произошло: обновление, крупный сбой или временное закрытие регистрации. Мы знаем, что игра была открыта раньше указанных дат, при этом с 22 по 27 октября инсталлов не было или почти не было.
+ 27 октября работа игры была восстановлена, появляются приемлемые значения событий инсталлов. К 27 октября был закуплен трафик, поэтому инсталлы совершаются как пользователями из органического, так и из неорганического трафика, при этом пользователей из неорганического трафика в первые дни в полтора-два раза больше.
+ Ко 2 ноября количество неорганического трафика сильно понизилось. Возможно, в первые три недели неорганический трафик периодически докупался, потому что 3 и 11 ноября был отмечен резкий рост количества инсталлов из неорганического трафика, примерно на 200%. С середины ноября по декабрь количество неорганического трафика значительно сократилось. 3 декабря было относительно большое увеличение инсталлов из неорганики (почти в 10 раз относительно прошлого дня, но всего 24% от общего количества инсталлов). После 4 декабря и на протяжении всего исследуемого периода количество инсталлов из неорганического трафика было незначительным.
+ Органический трафик был относительно стабилен с 27 октября по 10 ноября, количество установок 11 ноября резко выросло - на 300%. Ещё один пик роста числа органических установок был 22-23 марта (на 113% в первый и 130% на второй день).
+ Можно предположить, что 11 ноября 2020 года совершилось какое-то игровое событие, возможно, было выпущено крупное обновление. В этот день был значительный рост числа инсталлов как из органического, так и неорганического трафика. Первые пять дней после этого были наиболее значимы по количеству инсталлов за весь исследуемый период. До декабря наблюдалось снижение количества инсталлов, в первых числах декабря снова был период роста, но недолгий, к январю количество инсталлов снизилось и держалось примерно на одном уровне до 22 марта. При этом количество инсталлов было примерно таким же, как после возобновления деятельности игры 27 октября.

### Задача 2. Отличия динамики количества ежедневных инсталлов для разных регионов 

Есть ли отличия динамики количества ежедневных инсталлов  для разных регионов (СНГ, Европа, Азия, Северная Америка и Канада, Южная Америка)? Сделайте оценку в каких регионах более качественное удержание на начальном этапе (можно оценить, например, по второй сессии и проценту игроков, запустивших игру повторно).

#### Расчёты

In [ ]:
# Убираю дубликаты по пользователям, сохраняя страны:
country_installs = installs.drop_duplicates(['user_id']).reset_index(drop=True).loc[:, ('event_date', 'user_id', 'country')].reset_index(drop=True)
country_installs.head(2)

,event_date,user_id,country
0,2020-11-29,208124,BY
1,2020-11-29,136603,UA


In [ ]:
country_installs['country'].unique()

array(['BY', 'UA', 'RU', 'KZ', 'JP', 'UZ', 'PL', 'KG', 'MD', 'NL', 'GE',
       'BD', 'VN', 'TW', 'AZ', 'LV', 'AM', 'EE', 'KR', 'CZ', 'MM', 'TM',
       'EG', 'EC', 'US', 'CA', 'PR', 'MX', 'CR', 'DO', 'VI', 'NI', 'PA',
       None, 'GT', 'AI', 'TT', 'AG', 'KN', 'CU', 'JM', 'BB', 'HN', 'AW',
       'SV', 'PH', 'VE', 'DE', 'SA', 'UK', 'NG', 'BH', 'CN', 'LT', 'NO',
       'FR', 'ES', 'QA', 'BE', 'KE', 'IQ', 'SE', 'KW', 'BR', 'FI', 'RO',
       'BO', 'HK', 'BN', 'TH', 'HR', 'IL', 'CY', 'TR', 'SR', 'GU', 'CL',
       'AU', 'MY', 'AT', 'DK', 'TJ', 'ID', 'IR', 'GR', 'SY', 'IT', 'IN',
       'AR', 'AE', 'JO', 'BG', 'AF', 'MN', 'SG', 'UY', 'CO', 'PK', 'MG',
       'SK', 'SI', 'PS', 'ZA', 'PE', 'DZ', 'RE', 'GH', 'MV', 'RS', 'GW',
       'CH', 'LA', 'ME', 'NP', 'AL', 'AD', 'OM', 'HU', 'IE', 'PT', 'TN',
       'CM'], dtype=object)

Сгруппирую страны по категориям, чтобы получить более наглядную визуализацию в соответствии с поставленной задачей.

Меня интересуют следующие категории:
+ СНГ (CIS);
+ Европа (Europe);
+ Азия (Asia);
+ Северная Америка (North America);
+ Южная Америка (South America)

In [ ]:
def group_by_country(country):
    if country == 'RU' or country == 'UA' or country == 'BY' or country == 'AM' \
    or country == 'KZ' or country == 'UZ' or country == 'KG' or country == 'MD' \
    or country == 'AZ' or country == 'TJ':
        return 'CIS'
    elif country == 'PL' or country == 'LV' or country == 'DE' or country == 'LT' \
    or country == 'NO' or country == 'FR' or country == 'CZ' or country == 'ES' \
    or country == 'NL' or country == 'BE' or country == 'EE' or country == 'SE' \
    or country == 'FI' or country == 'RO' or country == 'HR' or country == 'AT' \
    or country == 'DK' or country == 'GR' or country == 'IT' or country == 'BG' \
    or country == 'SK' or country == 'SI' or country == 'RS' or country == 'CH' \
    or country == 'ME' or country == 'AL' or country == 'AD' or country == 'HU' \
    or country == 'IE' or country == 'PT':
        return 'Europe'
    elif country == 'PH' or country == 'GE' or country == 'JP' or country == 'SA' \
    or country == 'KR' or country == 'BH' or country == 'CN' or country == 'VN' \
    or country == 'QA' or country == 'IQ' or country == 'KW' or country == 'HK' \
    or country == 'BN' or country == 'TH' or country == 'MM' or country == 'IL' \
    or country == 'CY' or country == 'TR' or country == 'MY' or country == 'ID' \
    or country == 'IR' or country == 'SY' or country == 'IN' or country == 'AE' \
    or country == 'JO' or country == 'AF' or country == 'MN' or country == 'TM' \
    or country == 'SG' or country == 'PK' or country == 'TW' or country == 'PS' \
    or country == 'BD' or country == 'MV' or country == 'LA' or country == 'NP' \
    or country == 'OM':
        return 'Asia'
    elif country == 'US' or country == 'CA' or country == 'MX':
        return 'North America'
    elif country == 'VE' or country == 'BR' or country == 'BO' or country == 'SR' \
    or country == 'CL' or country == 'AR' or country == 'EC' or country == 'UY' \
    or country == 'CO' or country == 'PE':
        return 'South America'
#    elif country == 'PR' or country == 'CR' or country == 'DO' or country == 'VI' \
#    or country == 'NI' or country == 'PA' or country == 'GT' or country == 'AI' \
#    or country == 'TT' or country == 'AG' or country == 'KN' or country == 'CU' \
#    or country == 'JM' or country == 'BB' or country == 'HN' or country == 'AW' \
#    or country == 'SV':
#        return 'Central America'
#    elif country == 'NG' or country == 'EG' or country == 'KE' or country == 'MG' \
#    or country == 'ZA' or country == 'DZ' or country == 'RE' or country == 'GH' \
#    or country == 'GW' or country == 'TN' or country == 'CM':
#        return 'Africa'
    else:
        return 'Other'

country_installs['country_group'] = country_installs['country'].apply(group_by_country)


In [ ]:
# Группирую данные инсталлов по группе стран и дате:
country_installs = country_installs.groupby(['country_group', 'event_date'])['user_id'].nunique().reset_index().sort_values('event_date').reset_index(drop=True)

In [ ]:
# Переименовываю user_id в user_count:
country_installs.rename(columns={'user_id': 'user_count'}, inplace=True)

#### График

In [ ]:
# Получаю линейный график с визуализацией количества инсталлов пользователями из разных групп стран по датам:
fig = px.line(
    country_installs,
    x = 'event_date',
    y = 'user_count',
    color = 'country_group',
    title = 'Распределение инсталлов по группам стран',
)
fig.show()

#### Расчёты второго запуска

В имеющейся у нас базе данных первый вход с названием LaunchApp часто совпадает по времени с FirstLaunchApp. Иногда между этими событиями происходит 1 или несколько секунд. Делать расчёты повторного запуска на основании таких данных кажется не вполне объективным, поэтому я возьму события LaunchApp > FirstLauchApp по дате и оставлю среди них самые ранние, получив данные запусков второго дня.

In [ ]:
installs_date = installs.loc[:, ('event_name', 'event_date', 'event_time', 'user_id')].reset_index(drop=True)

In [ ]:
country_launches = project_dataset[project_dataset['event_name'] == 'LaunchApp'].reset_index(drop=True)

In [ ]:
country_launches['country_group'] = country_launches['country'].apply(group_by_country)

In [ ]:
country_launches = country_launches.loc[:, ('event_name', 'event_date', 'event_time', 'user_id', 'country_group')].reset_index(drop=True)

In [ ]:
country_launches_compare = pd.merge(installs_date, country_launches, how='left',on=['user_id'])

In [ ]:
good_launches = country_launches_compare[country_launches_compare['event_date_x'] < country_launches_compare['event_date_y']].reset_index(drop=True)

In [ ]:
second_day_launches = good_launches.groupby(['user_id', 'country_group', 'event_date_y']).agg({'event_time_y': 'min'}).reset_index().sort_values('event_time_y').reset_index(drop=True)

In [ ]:
second_day_launches = second_day_launches.groupby(['country_group', 'user_id']).agg({'event_date_y': min}).reset_index().sort_values('event_date_y').reset_index(drop=True) 

In [ ]:
second_day_launches = second_day_launches.groupby(['country_group', 'event_date_y']).agg({'user_id': 'nunique'}).reset_index().sort_values('event_date_y').reset_index(drop=True) 

In [ ]:
second_day_launches.rename(columns={'user_id': 'second_day_user_count', 'event_date_y': 'second_launch_day'}, inplace=True)

#### График

In [ ]:
# Получаю линейный график с визуализацией количества входов на второй день пользователями из разных групп стран по датам:
fig = px.line(
    second_day_launches,
    x = 'second_launch_day',
    y = 'second_day_user_count',
    color = 'country_group',
    title = 'Распределение количества запусков второго дня по группам стран',
)
fig.show()

#### Выводы

+ Сравнивая события инсталлов и второго дня запуска приложения, я могу сказать, что тенденции схожи среди всех регионов - периоды роста и снижения установок коррелируют с количеством повторных входов, при этом количество входов на следующий день равно примерно трети от количества установок в предыдущий день.
+ Ярко выделяются два основных региона - СНГ и Северная Америка. СНГ преобладает по количеству инсталлов и запусков.
+ Событие, повлекшее большое количество инсталлов 23 марта 2021 также вызвало пропорциональный рост повторных запусков 24 марта 2021. Это событие затронуло только игроков СНГ, данные по игрокам Северной Америки не изменились.
+ Обращаясь к более раннему анализу данных, отмечу, что, не смотря на большее количество пользователей из СНГ, пользователи из Северной Америки приносят почти в четыре раза больше прибыли. В связи с этим досадно выглядит тот факт, что всплеск активности в марте 2021 не затронул игроков этого региона.

#### Дополнительно

In [ ]:
second_day_launches.rename(columns={'second_launch_day': 'event_date', 'second_day_user_count': 'user_count'}, inplace=True)

In [ ]:
second_day_compare = pd.merge(country_installs, second_day_launches, how='inner',on=['event_date'])

In [ ]:
second_day_compare.rename(columns={'country_group_x': 'install_country_group', 'user_count_x': 'install_user_count',
                        'country_group_y': 'launch_country_group', 'user_count_y': 'launch_user_count'}, inplace=True)

In [ ]:
second_day_compare.head(2)

,install_country_group,event_date,install_user_count,launch_country_group,launch_user_count
0,North America,2020-10-28,236,Asia,1
1,North America,2020-10-28,236,Europe,1


In [ ]:
# Получаю график сравнения количества инсталлов и запусков второго дня по группам стран:
fig = px.line(
    second_day_compare,
    x = 'event_date',
    y = 'install_user_count',
    # y = 'launch_user_count',
    color = 'install_country_group',
    # color = 'launch_country_group',
    title = 'Распределение количества инсталлов по группам стран',
)
fig.show()

В будущем мне хотелось бы сделать выпадающий список, в котором было два варианта: инсталлы и запуски второго дня.
При выборе одного из этих вариантов менялось бы отображение на графике.
Было бы особенно наглядно, если при выборе одного варианта второй отображался пунктиром, чтобы можно было их сравнивать.